#### 1. Get together in groups as randomized here: https://docs.google.com/spreadsheets/d/1W0qAKkJ1_J6wXDOs0PhSTnWgwA9buZmMeI3YDGIYAYY/edit?usp=sharing

Group members:
- Luis Alejandro Quiñones.
- Dominik Johannes Wielath.
- Santiago Manotas-Arroyave.

#### 2. Download the material for the homework and get the booking scrape code to run. Prove that you did by attaching a jupyter notebook. (this is only relevant for those who fail later steps). 

Design and implement a mini research project in which you research the effect of a big annual event in Barcelona on rental prices on booking by scraping data for at least two separate weeks (note that search results go across different pages) for Barcelona and at least one more city. Below:

In [18]:
geko_path = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\geckodriver.exe'
dfolder = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\download'
link = 'https://www.booking.com/index.es.html'
download = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3'

In [37]:
#%%
#!pip3 install -r requirements.txt
import dropbox
from selenium.webdriver.common.keys import Keys
# auxiliary functions modified by Luis.
import scrape_functions as kzd
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np
import importlib
importlib.reload(sys.modules['scrape_functions'])
from datetime import date
# %% Starting code
geko_path = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\geckodriver.exe'


profile, browser, download_path = kzd.start_up(dfolder, link, download)
# Clicking on the different buttoms to input things:
time.sleep(3)

# Navigate to a webpage
browser.get("https://www.booking.com/index.es.html")
time.sleep(1)

# Click on  place
#%%
place = input('Where do you want to go?: ')
search1 = browser.find_element('xpath','//input[@placeholder="¿Where are you going?"]')
search1.send_keys(place)
#%%
#__bui-c969239-1
time.sleep(2)

# Click on dates:

kzd.check_and_click(
    browser, '//button[@class="d47738b911 e246f833f7 fe211c0731"]', type='xpath')

dates = browser.find_elements('xpath',
                              '//div[@class="bui-calendar__wrapper"]//table[@class="bui-calendar__dates"]//tbody//td[@class!="bui-calendar__date--empty"]')

from_day = input('Day from which you want to look for (In february): ')
#if int(from_day)< date.today().day:
    #print('Wrong from_day')

to_day = input('Day until which you want to look for (In february): ')

for date1 in dates:
    if date1.get_attribute("data-date") == f"2023-02-{from_day}":
        #date1.click()
        browser.execute_script("arguments[0].click();", date1)
    if date1.get_attribute("data-date") == f"2023-02-{to_day}":
        #date1.click()
        browser.execute_script("arguments[0].click();", date1)
        break


#%% Click to search:

kzd.check_and_click(
    browser, 'button.d4b6b7a9e7', type='css')
time.sleep(2)
# kzd.check_and_click(browser, 'div._fe1927d9e:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > h3:nth-child(1) > a:nth-child(1) > div:nth-child(1)', type='css')


hotels = browser.find_elements('xpath','//div[@class="fcab3ed991 a23c043802"]')
ratings = browser.find_elements('xpath','//div[@class="b5cd09854e d10a6220b4"]')
prices = browser.find_elements('xpath','//div[@class="fcab3ed991 fbd1d3018c e729ed5ab6"]')

ratings_list=[]
for i in ratings:
    #print(i.get_attribute('data-testid'))
    print(i.text)
    ratings_list.append(i.text)
hotels_list=[]
for i in hotels:
    #print(i.get_attribute('data-testid'))
    print(i.text)
    hotels_list.append(i.text)
prices_list=[]
for i in prices:
    #print(i.get_attribute('data-testid'))
    print(i.text)
    prices_list.append(i.text)
    

#%% LOOPING OVER WEBPAGES:

"""
This function is to obtain total number of pages.
To loop, just insert the full code on a big loop and use the pages output.


ADDITION: You can also add random waiting times to avoid getting banned from webpages.
"""

time.sleep(2)


def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    return(total_pages)


pages = get_number_pages(browser)
#From here it is very easy to program a loop that waits and keeps appending results

#
#
#

Where do you want to go?: Barcelona
Day from which you want to look for (In february): 2
Day until which you want to look for (In february): 5


WebDriverException: Message: Failed to decode response from marionette


#### 3. Identify an event that makes a lot people come to Barcelona.

Upcoming events in the calendar year (by order):
- Mobile World Congress, February.
- Festes de Santa Eulália, February.
- Barcelona Film Festival, April.
- Primavera Sound 2023, May/June.
- Sónar Festival 2023, June.
- La Mercé 2023, September.


#### 4. Think of the time periods to scrape and what second city to scrape. Explain your choices in written.

In order to establish a well-defined DiD regression we need to consider and comply for its assumptions. 

Firstly: Considering Barcelona as the main tourist hub for Spain during the summer season, the choice for another city would be initially narrowed down to cities in Spain that are also highly visited during these times. Along with keeping this in check, it would be wise to consider locations with similarities in what the city has to offer in the cultural and general entertainment scene; as well as population and extension factors. It is also important to denote that Barcelona is a coastal city, and thus we would also have to compare to a coastal city; or add a beach proximity control if choosing a landlocked sister location. 

The logical choice for a city that fulfills most of these conditions to match with Barcelona is Madrid (making up the 2 largest cities in Spain by most measures). Population-wise, the metropolitan areas have the closest numbers to each other; in terms of economic and cultural activities they are also the most comparable and concurred during peak seasons. Although Madrid is in the middle on the Spanish territory, inputting a control for coastal proximity would allow us to isolate for the effect we are looking for; i.e.: it makes a better sense to compare Barcelona and Madrid than Barcelona and Valencia, or even that with cities outside of Spain.

Secondly: The time periods to consider would be a comparable week (to consider as the "control" time period) and an event week. Ideally, the comparable week and the event week should have close similiraties in terms of seasonality, weather, and specially that they are close in time (as well as factors pertaining to other yearly events that might increase affluence to the city). Another thing to keep in mind is the scale of the affluency event. Given the cultural differences between regions, we have to consider between choosing an event that would attract differentc types of tourists. Corporate, topic-specific events from the likes of the Mobile World Congress may have less attendance figures than cultural celebrations such as La Mercé; or than music-central activities such as the Primavera Sound Festival.

Considering all of this, by attendance information of the past iterations of events mentioned, our pick is for the Primavera Sound 2023 (May 31st - June 4th). Out of past versions of events mentioned, it had the highest participation for last year, around ~450.000 estimated attendees.

#### 5. Write down a fixed effects regression equation that allows you to derive a difference-in-difference estimate of the effect of the event on prices. Think of controls to add, why is this relevant? Explain why you need a second city for this.

The bare fixed effects regression equation is:

$ Accommodation \ Price = \beta_{0} + \beta_{1} \cdot Event + \beta_{2} \cdot City + \beta_{3} \cdot Event \cdot City + \epsilon $

This equation however, does not include any controls, which may present omitted variable bias when running it like this. 

Differences-in-Differences approaches to causal inference require the assumptions of common trend and counterfactual trend for treated individuals (in the absence of treatment). The former property enables the underlying pattern of change that affects both cities, accounting for external factors. For the latter assumption, a counterfactual is requisite to draw comparisons of our target variable when the treatment (event) occurs for the group (city) we have decided to analyze. This in regard to a __comparable city__ where the event had not taken place.  Note that it is important for the correct set-up of a DiD (Differences-in-Differences) model that both subjects are comparable in underlying trends and characteristics; particularly those affecting economic and tourism patterns, which is why the best choice for Madrid would indeed be confirmed.

For a DiD estimator to correctly hold and for estimations to be arguably significant we would first have to account for underlying characteristics and trends that may be proxied by data we also pull. Controls such as seasonality (a dummy for high or low season), economic conditions (income, consumer spending, unemployment rate, inflation), weather controls (average temperature, precipitation), and (given what was mentioned in the last exercise) a beach (dummy) and event location proximity variable. These added variables are relevant in the sense that they isolate the effect we would be trying to estimate, which is the effect of the set event in Barcelona on accomoddation prices; reducing omitted variable bias.

$ Accommodation \ Price = \beta_{0} + \beta_{1} \cdot Event + \beta_{2} \cdot City + \beta_{3} \cdot Event \cdot City + \beta_{4} \cdot Economic + \beta_{5} \cdot Beach + \beta_{6} \cdot Location + \beta_{7} \cdot Seasonality + \epsilon $

#### 6. Scrape using a modified code that implements a loop across pages of the booking page and searches for different dates and cities.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
import scrape_functions as kzd
from selenium.webdriver import FirefoxOptions, Firefox, Chrome, ChromeOptions
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import date, datetime

Luis Paths:

In [5]:
# Start a new browser instance. Remember to change to your working directory
geko_path = '/Users/luisquinonespr/code/text_mining/pset3/HW3/geckodriver'
dfolder = '/Users/luisquinonespr/code/text_mining/pset3/HW3/resolutions'
link = 'https://www.booking.com/index.en.html'

Santi paths:

In [ ]:
geko_path = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\geckodriver.exe'
dfolder = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3\download'
link = 'https://www.booking.com/index.en.html'
download = r'C:\Users\santi\Documents\M_DSDM\Term_2\Text_Mining\HW3'

In [7]:
profile, browser, download_path = kzd.start_up(dfolder, link, download=True)

time.sleep(6)

#Inconsistent behavior, sometimes it will choose the drop down table, sometimes it will not, if it doesn't it will fail

kzd.check_and_click(
    browser, '//div[@class="b91c144835"]', type='xpath')
kzd.check_and_click(
    browser, '//div[@class="b91c144835"]', type='xpath')
kzd.check_and_click(
    browser, '//div[@class="b91c144835"]', type='xpath')

time.sleep(4)



months = {1: "January", 2: "February", 3:"March", 
               4:"April", 5:"May", 6:"June", 
               7:"July", 8:"August", 9:"September", 
               10:"October", 11:"November", 12:"December"}

# Months in display
month_label_xpath = '//h3[@class="ac78a73c96 ab0d1629e5"]'
next_month_button_xpath = '//button[@class="fc63351294 a822bdf511 e3c025e003 fa565176a8 cfb238afa1 ae1678b153 c9fa5fc96d be298b15fa"]'
month_in_display = False

def get_input(prompt, type_=int):
  while True:
    try:
      val = type_(input(prompt))
      if val > 0:
        return val
      else:
        print("Invalid input. Please try again.")
    except ValueError:
      print("Invalid input. Please try again.")

# Ask for start and end months and format them
start_month_number = get_input("Enter start month number (1-12): ", int)
if len(str(start_month_number)) < 2:
  start_month_number = start_month_number
end_month_number = get_input("Enter end month number (1-12): ", int)
if len(str(end_month_number)) < 2:
  end_month_number = end_month_number

# Check if the start and end months are displayed
start_month = months[start_month_number]
end_month = months[end_month_number]
month_in_display = False

while not month_in_display:
  displayed_months = browser.find_elements('xpath', month_label_xpath)
  displayed_months = list(map(lambda x: x.text.split(" ")[0], displayed_months))
  if start_month in displayed_months and end_month in displayed_months:
    month_in_display = True
  else:
    browser.find_element('xpath', next_month_button_xpath).click()

time.sleep(3)

dates=browser.find_elements('xpath',
    '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')

# Ask for start and end days and format them
from_day = get_input(f"Enter start day for {start_month} (1-31): ", int)
if len(str(from_day)) < 2:
  from_day = "0" + str(from_day)
to_day = get_input(f"Enter end day for {end_month} (1-31): ", int)
if len(str(to_day)) < 2:
  to_day = "0" + str(to_day)
if len(str(start_month_number)) < 2:
        start_month_number = "0" + str(start_month_number)
if len(str(end_month_number)) < 2:
        end_month_number = "0" + str(end_month_number)


# Select the dates
for date1 in dates:
    if date1.get_attribute("data-date") == f"2023-{start_month_number}-{from_day}":
        browser.execute_script("arguments[0].click();", date1)

dates=browser.find_elements('xpath',
    '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')

for date2 in dates:
    if date2.get_attribute("data-date") == f"2023-{end_month_number}-{to_day}":
        browser.execute_script("arguments[0].click();", date2)

# Define a function that handles the input and selection of destination

def select_destination(browser):
    place = input('Where do you want to go? ')
    search1 = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
    search1.send_keys(place)
# Wait for the options to appear
    time.sleep(2)
# Press the down arrow key to hover over the first option
    search1.send_keys(Keys.ARROW_DOWN)
# Press the enter key to select it
    search1.send_keys(Keys.RETURN)
    time.sleep(3)
    search1.send_keys(Keys.RETURN)

# Function that call the previous functions
def main(browser):
    select_destination(browser)
    time.sleep(3)

#Call the main function
main(browser)

kzd.check_and_click(
    browser, 'button.d4b6b7a9e7', type='css')

In [8]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    return(total_pages)

pages = get_number_pages(browser)

for page in range(pages) :

    hotels = browser.find_elements('xpath','//div[@class="fcab3ed991 a23c043802"]')
    ratings = browser.find_elements('xpath','//div[@class="b5cd09854e d10a6220b4"]')
    prices = browser.find_elements('xpath','//span[@class="fcab3ed991 fbd1d3018c e729ed5ab6"]')
    #locations = browser.find_elements('attr', '//address"]')


    hotel_dict={"hotel_name":[], "rating":[], "location":[],"price":[]}

    for i in ratings:
        print(i.text)
        hotel_dict['rating'].append(i.text)
    for i in hotels:
        print(i.text)
        hotel_dict['hotel_name'].append(i.text)
    for i in prices:
        hotel_dict['price'].append(i.text)

8.4
8.4
8.5
8.9
8.8
8.7
8.6
8.1
9.3
8.5
8.8
9.2
9.1
8.8
8.6
8.2
8.4
8.5
8.5
8.4
8.2
9.0
8.7
8.0
9.2
Vincci Bit
Sonder DO Plaça Reial
Lux Barcelona Apartment 5-Bedroom
TWO Hotel Barcelona by Axel 4* Sup- Adults Only
Ohla Eixample
Casa Elliot by Bondia Hotel Group
Hotel Paxton Barcelona
Ofelias Hotel 4* Sup
Acacia Suite
H10 Madison 4* Sup
Bonanova Suite
Hotel Well and Come
ME Barcelona
Hotel Brummell
Hostal Oliva
The Moods Catedral Hostal Boutique
Hotel Lleó
H10 Itaca
Gran Hotel Havana by Escampa Hotels
Catalonia Plaza Catalunya
Chic & Basic Lemon Boutique Hotel
Praktik Garden
Catalonia Passeig de Gràcia 4* Sup
Catalonia Portal de l'Angel
Hotel Lloret Ramblas
Praktik Èssens
8.4
8.4
8.5
8.9
8.8
8.7
8.6
8.1
9.3
8.5
8.8
9.2
9.1
8.8
8.6
8.2
8.4
8.5
8.5
8.4
8.2
9.0
8.7
8.0
9.2
Vincci Bit
Sonder DO Plaça Reial
Lux Barcelona Apartment 5-Bedroom
TWO Hotel Barcelona by Axel 4* Sup- Adults Only
Ohla Eixample
Casa Elliot by Bondia Hotel Group
Hotel Paxton Barcelona
Ofelias Hotel 4* Sup
Acacia Suite

In [9]:
df = pd.DataFrame.from_dict(hotel_dict, orient='index')
df = df.transpose()

df

,hotel_name,rating,location,price
0,Vincci Bit,8.4,None,"€ 3,487"
1,Sonder DO Plaça Reial,8.4,None,"€ 3,516"
2,Lux Barcelona Apartment 5-Bedroom,8.5,None,€ 435
3,TWO Hotel Barcelona by Axel 4* Sup- Adults Only,8.9,None,"€ 1,243"
4,Ohla Eixample,8.8,None,"€ 1,281"
5,Casa Elliot by Bondia Hotel Group,8.7,None,"€ 1,328"
6,Hotel Paxton Barcelona,8.6,None,"€ 1,311"
7,Ofelias Hotel 4* Sup,8.1,None,"€ 1,495"
8,Acacia Suite,9.3,None,"€ 1,360"
9,H10 Madison 4* Sup,8.5,None,"€ 2,579"


#### 7. Discuss two additional ideas (no need to implement): a) the use of the text that can be scraped from the hotel/apartment description b) how to de-construct the treatment effect in the effect that comes from the same hotel changing prices and composition of places changing.

- The use of text scraped from accommodation descriptions can be useful to denote price mark-ups in accordance to the ammenities that each accommodation has (think of high-speed internet, in-unit washing area, 24/7 surveillance, clerk availability, pool, gym). Hotel/apartment-quality/appeareance/architecture (i.e.: beautiful modern room, contemporary, loft-style) related adjectives may also be scraped to further differentiate between observations for more variability. It would be also important to consider through text scraping the unit descriptions if price premiums have a differential trend for different accommodation types; that is, varying estimators for hotel rooms, apartments, shared rooms, hostels.

- Intentional price changes from accommodation handlers are part of the estimation set out by the model, the presence of an event is surely a driver of price mark-ups around the time of the event. As discussed before, the presence of a comparison group similar in economic and demographic characteristics is imperative for the estimation. To overcome composition of places changing we can make use of data coming from rating pages, size of accommodation, budget-orientation, and type. Furthermore, making use of panel data to account for time-invariant differences between locations would get control over the second issue.